# K-Means Quiz
Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

We might want to take a look at the distribution of the Title+Body length feature we used before and instead of using the raw number of words create categories based on this length: short, longer,..., super long.

In the questions below I'll refer to length of the combined Title and Body fields as Description length (and by length we mean the number of words when the text is tokenized with pattern="\W").

In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, count, desc, explode, lit, min, max, split, stddev, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import RegexTokenizer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.clustering import KMeans
# TODOS: 
# 1) import any other libraries you might need
# 2) run the cells below to read the dataset and extract description length features
# 3) write code to answer the quiz questions

In [35]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

### Read Dataset

In [36]:
stack_overflow_data = 'Train_onetag_small.json'

In [37]:
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

### Build Description Length Features

In [38]:
df = df.withColumn("Desc", concat(col("Title"), lit(' '), col("Body")))

In [39]:
body_length = udf(lambda x: len(x), IntegerType())

In [40]:
regexTokenizer = RegexTokenizer(inputCol="Desc", outputCol="words", pattern="\\W")
df = regexTokenizer.transform(df)
df = df.withColumn("DescLength", body_length(df.words))

In [41]:
assembler = VectorAssembler(inputCols=["DescLength"], outputCol="DescVec")
df = assembler.transform(df)

# Question 1
How many times greater is the Description Length of the longest question than the Description Length of the shortest question (rounded to the nearest whole number)?

Tip: Don't forget to import Spark SQL's aggregate functions that can operate on DataFrame columns.

In [42]:
# TODO: write your code to answer this question
df.agg(min("DescLength")).show()

+---------------+
|min(DescLength)|
+---------------+
|             10|
+---------------+



In [43]:
df.agg(max("DescLength")).show()

+---------------+
|max(DescLength)|
+---------------+
|           7532|
+---------------+



# Question 2
What is the mean and standard deviation of the Description length?

In [44]:
# TODO: write your code to answer this question
df.agg(avg("DescLength"),stddev("DescLength")).show()

+---------------+-----------------------+
|avg(DescLength)|stddev_samp(DescLength)|
+---------------+-----------------------+
|      180.28187|     192.10819533505023|
+---------------+-----------------------+



# Question 3
Let's use K-means to create 5 clusters of Description lengths. Set the random seed to 42 and fit a 5-class K-means model on the Description length column (you can use KMeans().setParams(...) ). What length is the center of the cluster representing the longest questions?

In [45]:
# TODO: write your code to answer this question

kmeans = KMeans().setParams(featuresCol = "DescVec", predictionCol="DescGroup",k=5,seed=42)
model = kmeans.fit(df)
df = model.transform(df)

In [46]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', Desc="How to check if an uploaded file is an image without mime type? <p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an imag

In [47]:
df.groupby("DescGroup").agg(avg(col("DescLength")), avg(col("NumTags")), count(col("DescLength"))).orderBy("avg(DescLength)").show()

AnalysisException: "cannot resolve '`NumTags`' given input columns: [Desc, Tags, DescLength, Body, Title, DescGroup, DescVec, oneTag, words, Id];;\n'Aggregate [DescGroup#835], [DescGroup#835, avg(cast(DescLength#515 as bigint)) AS avg(DescLength)#886, avg('NumTags) AS avg(NumTags)#887, count(DescLength#515) AS count(DescLength)#888L]\n+- Project [Body#489, Id#490L, Tags#491, Title#492, oneTag#493, Desc#499, words#506, DescLength#515, DescVec#525, UDF(DescVec#525) AS DescGroup#835]\n   +- Project [Body#489, Id#490L, Tags#491, Title#492, oneTag#493, Desc#499, words#506, DescLength#515, UDF(named_struct(DescLength_double_VectorAssembler_71ba8eb8a062, cast(DescLength#515 as double))) AS DescVec#525]\n      +- Project [Body#489, Id#490L, Tags#491, Title#492, oneTag#493, Desc#499, words#506, <lambda>(words#506) AS DescLength#515]\n         +- Project [Body#489, Id#490L, Tags#491, Title#492, oneTag#493, Desc#499, UDF(Desc#499) AS words#506]\n            +- Project [Body#489, Id#490L, Tags#491